In [9]:
# 💬 Instalar AutoGluon si es necesario
#%pip install autogluon.timeseries

import pandas as pd
import numpy as np
from tqdm import tqdm
import gc, os, shutil
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
import time

In [10]:
def percentage_safe(numerator: pd.Series, denominator: pd.Series, dtype='float32', fillna=None) -> pd.Series:
    """
    Calcula un porcentaje seguro como numerator / denominator.
    - Reemplaza divisiones por cero o NaN con NaN.
    - Opcionalmente convierte a float32.
    - Puede rellenar NaNs con `fillna`.
    """
    result = (numerator / denominator).mask((denominator == 0) | (denominator.isna()))
    if fillna is not None:
        result = result.fillna(fillna)
    return result.astype(dtype)

In [11]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

In [12]:
df = pd.read_parquet("cliente_producto_base.parquet")

In [13]:
df = df.filter(items=['periodo', 'product_id', 'customer_id', 'tn'])

In [14]:
df.shape

(17173448, 4)

In [15]:
df[['periodo','product_id']].head(5)

,periodo,product_id
0,201701,20001
1,201701,20001
2,201701,20001
3,201701,20001
4,201701,20001


In [16]:
import pandas as pd
import time
# Asegúrate de tener las importaciones de la librería que usas (ej: autogluon)
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

# --- Carga de datos (asumimos que 'df' ya está cargado) ---
# df = pd.read_csv('tus_datos.csv') 
# df['periodo'] = df['periodo'].astype(str) # Asegurar que periodo es string

# --- INICIO DEL SCRIPT ---

df['timestamp'] = pd.to_datetime(df['periodo'], format='%Y%m')

# Combina las dos columnas para identificar la serie
df['item_id'] = df['product_id'].astype(str) + "_" + df['customer_id'].astype(str)

# Guarda los tipos de datos originales para usarlos después
original_types = {'product_id': df['product_id'].dtype, 'customer_id': df['customer_id'].dtype}

# 2. Lista de periodos ordenada
periods = sorted(df['timestamp'].unique())

# 4. Loop roll-forward
results = []
min_length = 7
start_idx = min_length + 2

for idx in range(start_idx, len(periods)):
    start_time = time.time()
    cutoff = periods[idx-2]
    target_period = periods[idx]
    cutoff_str = pd.to_datetime(cutoff).strftime('%Y%m')
    target_str = pd.to_datetime(target_period).strftime('%Y%m')

    print(f"\n🔷 Iteración {idx-start_idx+1}/{len(periods)-start_idx}")
    print(f"   📅 Cutoff: {cutoff_str}, Target: {target_str}")

    df_train = df[df['timestamp'] <= cutoff]

    counts = df_train.groupby('item_id').size()
    valid_ids = counts[counts >= min_length].index.tolist()
    if not valid_ids:
        print("   ⚠ No hay series con longitud suficiente, se salta esta iteración.")
        continue

    print(f"   ✅ Series válidas: {len(valid_ids)}")
    df_train = df_train[df_train['item_id'].isin(valid_ids)]
    n_meses = df_train['timestamp'].nunique()

    ts_train = TimeSeriesDataFrame.from_data_frame(
        df_train,
        id_column='item_id',
        timestamp_column='timestamp'
    ).fill_missing_values() # SUGERENCIA: Considera fill_missing_values(method='ffill') si no quieres rellenar con 0

    predictor = TimeSeriesPredictor(
        prediction_length=2,
        target='tn', # Asegúrate que la columna 'tn' exista en ts_train
        freq='MS'
    )
    print("   🚀 Entrenando predictor...")
    predictor.fit(ts_train, num_val_windows=2, time_limit=60*60)

    forecast = predictor.predict(ts_train)

    # CORRECCIÓN: Se elimina el mapeo redundante 'item_id': 'item_id'
    df_pred = forecast['mean'].reset_index().rename(columns={'mean': 'tn_pred_auto'})

    df_pred = df_pred[df_pred['timestamp'] == target_period]

    # Separa nuevamente product_id y customer_id
    df_pred[['product_id', 'customer_id']] = df_pred['item_id'].str.split('_', expand=True)

    # CORRECCIÓN (BUENA PRÁCTICA): Se evita inplace=True
    df_pred = df_pred.drop(columns=['item_id'])

    # CORRECCIÓN (CRÍTICA): Convierte las columnas a su tipo de dato original
    df_pred = df_pred.astype({'product_id': original_types['product_id'],
                              'customer_id': original_types['customer_id']})

    df_pred['n_meses_hist'] = n_meses
    df_pred['periodo_pred'] = cutoff_str
    
    # La selección de columnas final es una excelente práctica
    results.append(df_pred[['product_id', 'customer_id', 'periodo_pred', 'timestamp', 'tn_pred_auto', 'n_meses_hist']])
    
    elapsed = time.time() - start_time
    print(f"   ⏱ Iteración completada en {elapsed:.1f} segundos.")

# Al final, puedes concatenar todos los resultados en un único DataFrame
final_results_df = pd.concat(results, ignore_index=True)



🔷 Iteración 1/27
   📅 Cutoff: 201708, Target: 201710
   ✅ Series válidas: 374316


Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Maestria Ciencia de Datos\Labo 3\TP\Dataset\AutogluonModels\ag-20250714_011534'


   🚀 Entrenando predictor...


=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.22
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       3.73 GB / 15.69 GB (23.7%)
Disk Space Avail:   142.33 GB / 459.95 GB (30.9%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}

Provided train_data has 2950591 rows, 374316 time series. Median time series length is 8 (min=7, max=8). 
Time series in train_data are too short for chosen num_val_windows=2. Reducing num_val_windows to 1.

Provided data contains following col

   ⏱ Iteración completada en 1130.6 segundos.

🔷 Iteración 2/27
   📅 Cutoff: 201709, Target: 201711
   ✅ Series válidas: 395357


...

   ⏱ Iteración completada en 1767.0 segundos.

🔷 Iteración 27/27
   📅 Cutoff: 201910, Target: 201912
   ✅ Series válidas: 600305


Beginning AutoGluon training... Time limit = 3600s
AutoGluon will save models to 'c:\Maestria Ciencia de Datos\Labo 3\TP\Dataset\AutogluonModels\ag-20250714_131647'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.22
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
GPU Count:          0
Memory Avail:       5.76 GB / 15.69 GB (36.7%)
Disk Space Avail:   117.66 GB / 459.95 GB (25.6%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 2,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'time_limit': 3600,
 'verbosity': 2}



   🚀 Entrenando predictor...


Provided train_data has 15708766 rows, 600305 time series. Median time series length is 32 (min=7, max=34). 
	Removing 44625 short time series from train_data. Only series with length >= 9 will be used for training.
	After filtering, train_data has 15377596 rows, 555680 time series. Median time series length is 33 (min=9, max=34). 

Provided data contains following columns:
	target: 'tn'
	past_covariates:
		categorical:        []
		continuous (float): ['periodo', 'product_id', 'customer_id']

To learn how to fix incorrectly inferred types, please see documentation for TimeSeriesPredictor.fit

AutoGluon will gauge predictive performance using evaluation metric: 'WQL'
	This metric's sign has been flipped to adhere to being higher_is_better. The metric score can be multiplied by -1 to get the metric value.

Starting training. Start time is 2025-07-14 10:18:10
Models that will be trained: ['SeasonalNaive', 'RecursiveTabular', 'DirectTabular', 'NPTS', 'DynamicOptimizedTheta', 'AutoETS', 'Ch

   ⏱ Iteración completada en 1754.0 segundos.


In [17]:
df_all_preds = pd.concat(results, ignore_index=True)

In [18]:
df_all_preds.head()

,product_id,customer_id,periodo_pred,timestamp,tn_pred_auto,n_meses_hist
0,20001,10001,201708,2017-10-01,75.478121,8
1,20001,10002,201708,2017-10-01,20.881114,8
2,20001,10003,201708,2017-10-01,80.516141,8
3,20001,10004,201708,2017-10-01,147.217463,8
4,20001,10005,201708,2017-10-01,14.255742,8


In [19]:
df_all_preds['periodo'] = df_all_preds['timestamp'].dt.strftime('%Y%m').astype(int)

In [50]:
df['customer_id']  = df['customer_id'].astype(np.int32)
df['product_id']   = df['product_id'].astype(np.int32)
df['periodo']      = df['periodo'].astype(np.int32)

df_all_preds['customer_id'] = df_all_preds['customer_id'].astype(np.int32)
df_all_preds['product_id']  = df_all_preds['product_id'].astype(np.int32)
df_all_preds['periodo']     = df_all_preds['periodo'].astype(np.int32)


In [22]:
del df_all_preds['periodo_pred']
del df_all_preds['timestamp'] 

In [39]:
df_all_preds.head(5)

,product_id,customer_id,tn_pred_auto,n_meses_hist,periodo
0,20001,10001,75.478121,8,201710
1,20001,10002,20.881114,8,201710
2,20001,10003,80.516141,8,201710
3,20001,10004,147.217463,8,201710
4,20001,10005,14.255742,8,201710


In [23]:
df_all_preds.to_csv('predicciones_autogluon.csv', index=False)

In [24]:
df = pd.read_parquet("cliente_producto_base.parquet")

In [52]:
df=reduce_mem_usage(df)

Uso de memoria inicial del DataFrame: 5635.93 MB


C:\Users\diana\AppData\Local\Temp\ipykernel_24920\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


Uso de memoria final del DataFrame: 5574.67 MB
Memoria reducida en un 1.09%


In [53]:
df_all_preds=reduce_mem_usage(df_all_preds)

Uso de memoria inicial del DataFrame: 314.44 MB
Uso de memoria final del DataFrame: 123.53 MB
Memoria reducida en un 60.71%


In [ ]:
df = df.merge(
   df_all_preds,
    on=['customer_id','product_id','periodo'],
    how='left'
)

In [26]:
df.head()

,customer_id,product_id,periodo,cust_request_qty,cust_request_tn,tn,cliente_activo,producto_activo,inicio_vida_c,inicio_vida_p,...,cliente_total_delta_min_movil_6,cliente_total_delta_min_movil_12,cliente_total_delta_std_movil_3,cliente_total_delta_std_movil_,cliente_total_delta_std_movil_12,cliente_trend,cliente_season_yearly,clase,tn_pred_auto,n_meses_hist
0,10001,20001,201701,11,99.437500,99.437500,1.0,1.0,201701,201701,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.0625,NaN,NaN
1,10002,20001,201701,17,38.687500,35.718750,1.0,1.0,201701,201701,...,3144.0,3144.0,3140.0,3142.0,2512.0,NaN,NaN,149.0000,NaN,NaN
2,10003,20001,201701,17,143.500000,143.500000,1.0,1.0,201701,201701,...,2424.0,2426.0,1623.0,1132.0,1468.0,NaN,NaN,-124.4375,NaN,NaN
3,10004,20001,201701,9,184.750000,184.750000,1.0,1.0,201701,201701,...,1738.0,1738.0,878.0,1127.0,1306.0,NaN,NaN,-140.8750,NaN,NaN
4,10005,20001,201701,23,19.078125,19.078125,1.0,1.0,201701,201701,...,1059.0,1059.0,1058.0,1056.0,663.0,NaN,NaN,46.0625,NaN,NaN


In [27]:
df['tn_pred_auto_delta_a_tn']=df['tn_pred_auto']-df['tn'].fillna(0)
df['ratio_tn_pred_auto_delta_a_tn']=percentage_safe(df['tn_pred_auto_delta_a_tn'],df['tn'])
df['ratio_tn_pred_a_tn']=percentage_safe(df['tn_pred_auto'],df['tn'])

df=reduce_mem_usage(df)

Uso de memoria inicial del DataFrame: 6190.84 MB


C:\Users\diana\AppData\Local\Temp\ipykernel_24920\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


Uso de memoria final del DataFrame: 5830.53 MB
Memoria reducida en un 5.82%


In [37]:
df[['periodo','product_id','customer_id','tn','clase','tn_pred_auto','tn_pred_auto_delta_a_tn','ratio_tn_pred_auto_delta_a_tn','ratio_tn_pred_a_tn','tn','n_meses_hist']].head(100)

,periodo,product_id,customer_id,tn,clase,tn_pred_auto,tn_pred_auto_delta_a_tn,ratio_tn_pred_auto_delta_a_tn,ratio_tn_pred_a_tn,tn,n_meses_hist
0,201701,20001,10001,99.437500,44.062500,NaN,NaN,NaN,NaN,99.437500,NaN
1,201701,20001,10002,35.718750,149.000000,NaN,NaN,NaN,NaN,35.718750,NaN
2,201701,20001,10003,143.500000,-124.437500,NaN,NaN,NaN,NaN,143.500000,NaN
3,201701,20001,10004,184.750000,-140.875000,NaN,NaN,NaN,NaN,184.750000,NaN
4,201701,20001,10005,19.078125,46.062500,NaN,NaN,NaN,NaN,19.078125,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,201701,20001,10097,0.615723,-0.246338,NaN,NaN,NaN,NaN,0.615723,NaN
96,201701,20001,10098,3.939453,-3.570312,NaN,NaN,NaN,NaN,3.939453,NaN
97,201701,20001,10099,0.369385,-0.369385,NaN,NaN,NaN,NaN,0.369385,NaN
98,201701,20001,10101,0.369385,-0.369385,NaN,NaN,NaN,NaN,0.369385,NaN


In [38]:
df[['tn_pred_auto']].value_counts()

NotImplementedError: float16 indexes are not supported

In [29]:
df.to_parquet('0_b_cliente_producto_autogluon_completo.parquet', index=False)

In [ ]:
#df_train = df[~df['periodo'].isin([201911, 201912])]
df_train = df.query("periodo != 201911 and periodo != 201912")

df_train.to_parquet('0_b_cliente_producto_autogluon_train.parquet', index=False)
print(f"DataFrame de entrenamiento guardado en 'train.parquet' con {len(df_train)} filas.")

# --- 3. Preparar y guardar el DataFrame de predicción en Parquet ---
# Seleccionamos los periodos 201911 y 201912 para el conjunto de predicción.
# Eliminamos la columna 'clase' ya que no será necesaria para la predicción.
# Finalmente, guardamos este DataFrame en un archivo Parquet.
df_predecir = df[df['periodo'].isin([201911, 201912])].copy() # Usar .copy() para evitar SettingWithCopyWarning
df_predecir.drop(columns=['clase'], inplace=True)
df_predecir.to_parquet('0_b_cliente_producto_autogluon_predecir.parquet', index=False)
print(f"DataFrame para predicción guardado en 'predecir.parquet' con {len(df_predecir)} filas.")